In [1]:
import argparse
import logging
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torch.nn.functional as F

# from torch.utils.tensorboard import SummaryWriter

from utils import data
import models, utils

import pandas as pd
from laspy.file import File
from pickle import dump, load

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as udata
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

import lidar_data_processing

%matplotlib inline

In [2]:
class Args(object):
    def __init__(self):
        self.data_path= 'data' # not used
        self.dataset= 'masked_pwc' # move lidar into datasets
        self.batch_size= 32
        self.model= 'lidar_unet2d'
        self.in_channels = 7
        self.lr= 0.005
        self.weight_decay = 0.005
        self.num_epochs= 20
        self.n_data = 1 # not used
        self.min_sep = 5 # not used
        self.num_scan_lines = 300
        self.seq_len = 64
        self.scan_line_gap_break = 7000 # threshold over which scan_gap indicates a new scan line
        self.min_pt_count = 1700 # in a scan line, otherwise line not used
        self.max_pt_count = 2000 # in a scan line, otherwise line not used
        self.mask_pts_per_seq = 5
        self.mask_consecutive = True
        # points in between scan lines
        self.stride_inline = 5
        self.stride_across_lines = 3
        self.valid_interval= 1 
        self.save_interval= 1
        self.seed = 0
        self.output_dir= 'lidar_experiments/2d'
        self.experiment= None
        self.resume_training= False
        self.restore_file= None
        self.no_save= False
        self.step_checkpoints= False
        self.no_log= False
        self.log_interval= 100
        self.no_visual= False
        self.visual_interval= 100
        self.no_progress= False
        self.draft= False
        self.dry_run= False
        self.bias= False 
#         self.in_channels= 1 # maybe 6?
        self.test_num = 0
        # UNET
        self.residual = False
        self.wtd_loss = True
args=Args()

In [3]:
# gpu or cpu
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# args = utils.setup_experiment(args)
# utils.init_logging(args)

### Load the pts files

In [4]:
# Loads as a list of numpy arrays
scan_line_tensor = torch.load('../lidar_data/'+'scan_line_tensor.pts')
train_idx_list = torch.load('../lidar_data/'+'train_idx_list.pts')
valid_idx_list = torch.load('../lidar_data/'+'valid_idx_list.pts')
sc = torch.load('../lidar_data/'+'sc.pts')

In [5]:
def add_mask(sample,mask_pts_per_seq,consecutive=True):
    # Given a 3-D tensor of all ones, returns a mask_tensor of same shape 
    # with random masking determined by mask_pts_per_seq
    mask_tensor = torch.ones(sample.shape)
    seq_len = mask_tensor.shape[0]
    
    if consecutive:
        # Creates a square of missing points
        first_mask_row = int(np.random.choice(np.arange(8,seq_len-8-mask_pts_per_seq),1))
        first_mask_col = int(np.random.choice(np.arange(8,seq_len-8-mask_pts_per_seq),1))
        
        mask_tensor[first_mask_row:first_mask_row+mask_pts_per_seq,first_mask_col:first_mask_col+mask_pts_per_seq,:] = 0
            
    else:
        # TO DO: Random points throughout the patch
        for i in range(sample.shape[0]):
            m[i,:] = np.random.choice(np.arange(8,seq_len-8),mask_pts_per_seq,replace=False)

    return mask_tensor

In [6]:
# Dataloader class
class LidarLstmDataset(udata.Dataset):
    def __init__(self, scan_line_tensor, idx_list, seq_len = 64, mask_pts_per_seq = 5, consecutive = True):
        super(LidarLstmDataset, self).__init__()
        self.scan_line_tensor = scan_line_tensor
        self.idx_list = idx_list
        self.seq_len = seq_len
        self.mask_pts_per_seq = mask_pts_per_seq
        self.consecutive = consecutive

    def __len__(self):
        return len(self.idx_list)

    def __getitem__(self,index):
        row = self.idx_list[index][0]
        col = self.idx_list[index][1]
        clean = self.scan_line_tensor[row:row+self.seq_len,col:col+self.seq_len,:]
        mask = add_mask(clean,self.mask_pts_per_seq,self.consecutive)
        return clean.permute(2,0,1), mask.permute(2,0,1)

In [7]:
seq_len = 64
mask_pts_per_seq = 5
train_dataset = LidarLstmDataset(scan_line_tensor,train_idx_list,seq_len, mask_pts_per_seq)
valid_dataset = LidarLstmDataset(scan_line_tensor,valid_idx_list,seq_len, mask_pts_per_seq)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, num_workers=4, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, num_workers=4, shuffle=True)

In [8]:
c,m = next(iter(train_loader))
c.shape

torch.Size([32, 6, 64, 64])

## 2D interpolation

In [9]:
def weighted_MSELoss(pred,true,sc,mask_pts_per_seq=args.mask_pts_per_seq):
    '''weighted_MSELoss reconverts MSE loss back to the original scale of x,y,z.
    Rationale is because xyz have such different ranges, we don't want to ignore the ones with largest scale.
    Assumes that x,y,z are the first 3 features in sc scaler'''
    
    ranges = torch.Tensor(sc.data_max_[:3]-sc.data_min_[:3])
    raw_loss = torch.zeros(3,dtype=float)
    for i in range(3):
        raw_loss[i] = F.mse_loss(pred[:,i,:,:], true[:,i,:,:], reduction="sum") 
    return (ranges**2 * raw_loss).sum()

In [10]:
n = 2
out = lidar_data_processing.outer_interp_loop(c,m,mask_pts_per_seq,n)
weighted_MSELoss(out,c,sc,mask_pts_per_seq) / (c.shape[0]*mask_pts_per_seq**2)

tensor(444.5171, dtype=torch.float64)